### Fuzzy matching demo
Fuzzywuzzy package
- Uses Levenshtein distance to measure the similarity between two string
- library [link](https://anaconda.org/jkroes/fuzzywuzzy)
- Referece code, [link](https://medium.com/better-programming/fuzzy-string-matching-with-python-cafeff0d29fe)

In [8]:
from fuzzywuzzy import process, fuzz
import pandas as pd
from tqdm.notebook import tqdm, trange

#### Simple ratio
The `ratio` method compares the whole string and follows the standard Levenshtein distance similarity ratio between two strings:

In [20]:
# Simple ratio
String_Matched = fuzz.ratio('Hello World', 'Hello World!')
print("String Matched:",String_Matched)
String_Matched = fuzz.ratio('Hello World', 'Hello world')
print("String Matched:",String_Matched)
String_Matched = fuzz.ratio('Hello world', 'Hello world')
print("String Matched:",String_Matched)

String Matched: 96
String Matched: 91
String Matched: 100


#### Partial ratio
The `partial_ratio` method works on “optimal partial” logic. If the short string k and long string m are considered, the algorithm will score by matching the length of the k string:

In [23]:
# Partial ratio
Str_Partial_Match = fuzz.partial_ratio('Hello World', 'Hello World!')
print("String Matched:",Str_Partial_Match)
Str_Partial_Match = fuzz.partial_ratio('Hello World', 'Hello world')
print("String Matched:",Str_Partial_Match)

String Matched: 100
String Matched: 91


#### Token sort ratio
The `token_sort_ratio` method sorts the tokens alphabetically. Then, the simple `ratio` method is applied to output the matched percentage:

In [24]:
# Token sort ratio
Str_Sort_Match = fuzz.token_sort_ratio('Hello World', 'Hello wrld')
print("String Matched:",Str_Partial_Match)
Str_Sort_Match = fuzz.token_sort_ratio('Hello World', 'world Hello')
print("String Matched:",Str_Partial_Match)

String Matched: 91
String Matched: 91


#### Token set ratio
The `token_set_ratio` ignores the duplicate words. It is similar to the sort ratio method but more flexible. It basically extracts the common tokens and then applies `fuzz.ratio()` for comparisons:

In [ ]:
# Token set ratio
String_Matched=fuzz.token_set_ratio('Hello World', 'Hello Hello world')
print(String_Matched)

## Example
**Description:** Given a company name, find its standard name and stock tickers. 
**Context:** It is quite often we need to find out if a company public or private and its stock tickers. Quite often, the name we have is not standard, like "Apple" , "apple inc". We need to compare the name to NYSE/NASDAQ/AMEX/OTC data and fuzzy matching it to the right company and estimate the quality of the matching. 


In [4]:

# Source of tickers: https://www.nasdaq.com/market-activity/stocks/screener?exchange=NASDAQ&render=download
# This is the dictionary
nasdaq_tickers = pd.read_csv('data/NASDAQ.csv') # read Nasdaq company list
amex_tickers = pd.read_csv('data/AMEX.csv') # read Amex company list
otcbb_tickers = pd.read_csv('data/OTCBB.csv') # read OTC company list
nyse_tickers = pd.read_csv('data/NYSE.csv') # read NYSE company list

# Add exchange label
nasdaq_tickers['exchange'] = 'Nasdaq'
amex_tickers['exchange'] = 'AMEX'
otcbb_tickers['exchange'] = 'OTC'
nyse_tickers['exchange'] = 'NYSE'

# Read the list of companies which we need look for tickers
df = pd.read_csv('data/CES2021.csv')
df.columns = ['company']
df.head()

,company
0,1drop Inc.
1,"1VALET, Inc."
2,2020 Companies
3,233621
4,"247mytutor Co., Ltd."


In [5]:
# Union all tickers together
tickers = pd.concat([nasdaq_tickers, amex_tickers, otcbb_tickers, nyse_tickers])
tickers.reset_index(drop=True, inplace=True)
tickers = tickers.astype(str)
tickers.head()

,Symbol,Name,exchange
0,AACG,Ata Creativity Global,Nasdaq
1,AACQ,Artius Acquisition Inc Cl A,Nasdaq
2,AACQU,Artius Acquisition Inc Unit,Nasdaq
3,AACQW,Artius Acquisition Inc WT,Nasdaq
4,AAL,American Airlines Gp,Nasdaq


#### Function to match tickers to a list of companies
`tickers` is a dictionary of company name and stock tickers. Our goal is to get the sotck ticker by fuzzy matching standard company name to `stra` in dataframe `df`.

In [6]:
# Function searching ticker list and find the top 3 similar record
# a_str: string to be matched
# df: dictionary
# b_str: target
# b_return: a list of columns to be returned when matched. 
def fuzzy_match(a_str, df, b_str, b_return, topN = 1):
    df['score'] = df.apply(lambda row : fuzz.token_set_ratio(a_str.lower(), 
                                row[b_str].lower()), axis = 1)
    # get the top record by similarity
    top_record = df.nlargest(topN, 'score')[[b_str]+ b_return+ ['score']]
    return top_record.iloc[0]
# Test
print(fuzzy_match('Big 5 Sporting Goods', tickers, 'Name' ,['Symbol','exchange']))

Name        Big 5 Sporting
Symbol                BGFV
exchange            Nasdaq
score                  100
Name: 427, dtype: object


In [ ]:
# There are 2000 companied to be matched and 20k tickers in the dictionary. It takes ~ 5 min to complete
# Create and register a new `tqdm` instance with `pandas`
tqdm.pandas()
# Use progress_apply() instead of apply to call the progress bar
df[['name','ticker','exchange','score']] = df.progress_apply(lambda row : fuzzy_match(row['company'], tickers, 'Name' ,['Symbol','exchange']), axis = 1)

/Users/yan/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [72]:
df
df.to_csv('data/ces2021_matched.csv', index = False)